In [1]:
import pandas as pd
import scipy.stats as stats
import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols
import os
import ast
import numpy as np
import matplotlib.pyplot as plt


# Analysis <font color=blue>after </font> combining RF, DT and SVM into 1C 10 min

In [2]:
def get_class (model):
    if isinstance(model, str):
        model = '{' +'}'.join('{'.join(model.split('{')[1:]).split('}')[0:1]) + '}'
        m = ast.literal_eval(model)
        if 'classifier:__choice__' in m:
            classifier = m['classifier:__choice__']
        else:
            classifier = ''
        return classifier
    else:
        return ''


def parse_tpot(directory):
    result = pd.DataFrame(columns=['dataset', 'accuracy', 'model', 'precision', 'recall', 'f1score', 'time_budget', 'methods'])
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                method = file.split('.')[0].split('_')[2]
                time_budget = file.split('.')[0].split('_')[3]
                run = []
                sub_result = pd.read_csv(os.path.join(subdir, file))
                sub_result.rename(columns={'Unnamed: 0': 'dataset'}, inplace = True)
                run.append(sub_result[['dataset', 'accuracy_1', 'model_1', 'precision_1', 'recall_1', 'f1score_1']])
                run.append(sub_result[['dataset', 'accuracy_2', 'model_2', 'precision_2', 'recall_2', 'f1score_2']])
                run.append(sub_result[['dataset', 'accuracy_3', 'model_3', 'precision_3', 'recall_3', 'f1score_3']])
                for i in range(3):
                    run[i].rename(columns={'accuracy_'+str(i+1): 'accuracy',
                                           'model_'+str(i+1): 'model',
                                           'precision_'+str(i+1): 'precision',
                                           'recall_'+str(i+1): 'recall',
                                           'f1score_'+str(i+1): 'f1score'}, inplace=True)
                    run[i]['methods'] = str(method)
                    run[i]['time_budget'] = int(time_budget)
                    result = pd.concat([result, run[i]], axis=0, sort=True, ignore_index=True)
    result.model = result.model.apply(get_class)
    result = result[~np.isnan(result.f1score)]
    return result
#parse_tpot(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/")

In [3]:
df = parse_tpot(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/")
df = df[['dataset', 'time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("default", "fc")
df.methods = df.methods.replace("3C", "3c")
df.methods = df.methods.replace("SVC", "1c")
df.methods = df.methods.replace("DT", "1c")
df.methods = df.methods.replace("RF", "1c")
fsIs_b = ['vowel', 'openml_phpJNxH0q', 'dataset_31_credit-g', 'dataset_40_sonar']
fsIs_m = ['solar-flare_1', 'wine-quality-red', 'dataset_39_ecoli', 'synthetic_control']
fsIl_b = ['AirlinesCodrnaAdult', 'MagicTelescope', 'electricity-normalized', 'phpmPOD5A']
fsIl_m = ['pokerhand-normalized', 'eye_movements', 'avila-tr']
flIs_b = ['audiology', 'arrhythmia', 'AP_Breast_Lung', 'AP_Omentum_Ovary']
flIs_m = ['Amazon', 'umistfacescropped', 'phpGUrE90']
flIl_b = ['gina_agnostic', 'hiva_agnostic', 'phpZrCzJR', 'phprAeXmK']
flIl_m = ['KDDCup99', 'connect-4', 'dataset_60_waveform-5000', 'dataset_186_satimage']
df_binary = df[df.dataset.isin(fsIs_b) | df.dataset.isin(fsIl_b) | df.dataset.isin(flIs_b) | df.dataset.isin(flIl_b)]
df_multi = df[df.dataset.isin(fsIs_m) | df.dataset.isin(fsIl_m) | df.dataset.isin(flIs_m) | df.dataset.isin(flIl_m)]
#df.drop(df[(df.methods=='fc') & ((df.time_budget==30) | (df.time_budget==10))].index, inplace=True)
#df.drop(df[(df.methods=='3c') & ((df.time_budget==60) | (df.time_budget==10))].index, inplace=True)
df=df_multi
df10 = df[df.time_budget==10]
df30 = df[df.time_budget==30]
df60 = df[df.time_budget==60]

df.shape

C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

(181, 4)

In [4]:
rp.summary_cont(df10['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,52.0,0.696551,0.218529,0.030305,0.635712,0.75739


In [5]:
rp.summary_cont(df30['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,61.0,0.666992,0.215246,0.027559,0.611864,0.722119


In [6]:
rp.summary_cont(df60['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,68.0,0.68533,0.221225,0.026828,0.631782,0.738878


In [7]:
stats.f_oneway(df10['f1score'][df10['methods'] == '1c'], 
             df10['f1score'][df10['methods'] == '3c'],
             df10['f1score'][df10['methods'] == 'fc'])

F_onewayResult(statistic=1.243667555981208, pvalue=0.2972650245834688)

In [8]:
stats.f_oneway(df30['f1score'][df30['methods'] == '1c'], 
             df30['f1score'][df30['methods'] == '3c'],
             df30['f1score'][df30['methods'] == 'fc'])

F_onewayResult(statistic=2.5945857291723238, pvalue=0.08332487049392728)

In [9]:
stats.f_oneway(df60['f1score'][df60['methods'] == '1c'], 
             df60['f1score'][df60['methods'] == '3c'],
             df60['f1score'][df60['methods'] == 'fc'])

F_onewayResult(statistic=4.345806860786197, pvalue=0.016927659527741397)

In [10]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model10 = ols('f1score ~ C(methods)', df10).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model10.df_model: .0f},{model10.df_resid: .0f}) = {model10.fvalue: .3f}, p = {model10.f_pvalue: .4f}")
    model10.summary()

Overall model F( 2, 49) =  1.244, p =  0.2973


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     1.244
Date:                Fri, 01 Nov 2019   Prob (F-statistic):              0.297
Time:                        22:28:56   Log-Likelihood:                 7.0909
No. Observations:                  52   AIC:                            -8.182
Df Residuals:                      49   BIC:                            -2.328
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.6596      0.038     17.156      0.000       0.582       0.737
C(methods)[T.3c]     0.1105      0.079      1.402      0.167      -0.048       0.269
C(methods)[T.fc]     0.0816      0.079      1.035      0.306      -0.077       0.240
==============================================================================
Omnibus:                        0.401   Durbin-Watson:                   2.232
Prob(Omnibus):                  0.819   Jarque-Bera (JB):                0.561
Skew:                          -0.073   Prob(JB):                        0.756
Kurtosis:                       2.513   Cond. No.                         3.15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model30 = ols('f1score ~ C(methods)', df30).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model30.df_model: .0f},{model30.df_resid: .0f}) = {model30.fvalue: .3f}, p = {model30.f_pvalue: .4f}")
    model30.summary()

Overall model F( 2, 58) =  2.595, p =  0.0833


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                  0.050
Method:                 Least Squares   F-statistic:                     2.595
Date:                Fri, 01 Nov 2019   Prob (F-statistic):             0.0833
Time:                        22:28:56   Log-Likelihood:                 10.257
No. Observations:                  61   AIC:                            -14.51
Df Residuals:                      58   BIC:                            -8.181
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.6144      0.035     17.330      0.000       0.543       0.685
C(methods)[T.3c]     0.1321      0.075      1.756      0.084      -0.018       0.283
C(methods)[T.fc]     0.1179      0.063      1.863      0.068      -0.009       0.245
==============================================================================
Omnibus:                        0.103   Durbin-Watson:                   2.086
Prob(Omnibus):                  0.950   Jarque-Bera (JB):                0.148
Skew:                           0.089   Prob(JB):                        0.929
Kurtosis:                       2.836   Cond. No.                         3.25
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model60 = ols('f1score ~ C(methods)', df60).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model60.df_model: .0f},{model60.df_resid: .0f}) = {model60.fvalue: .3f}, p = {model60.f_pvalue: .4f}")
    model10.summary()

Overall model F( 2, 65) =  4.346, p =  0.0169


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     1.244
Date:                Fri, 01 Nov 2019   Prob (F-statistic):              0.297
Time:                        22:28:56   Log-Likelihood:                 7.0909
No. Observations:                  52   AIC:                            -8.182
Df Residuals:                      49   BIC:                            -2.328
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.6596      0.038     17.156      0.000       0.582       0.737
C(methods)[T.3c]     0.1105      0.079      1.402      0.167      -0.048       0.269
C(methods)[T.fc]     0.0816      0.079      1.035      0.306      -0.077       0.240
==============================================================================
Omnibus:                        0.401   Durbin-Watson:                   2.232
Prob(Omnibus):                  0.819   Jarque-Bera (JB):                0.561
Skew:                          -0.073   Prob(JB):                        0.756
Kurtosis:                       2.513   Cond. No.                         3.15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

mc = MultiComparison(df10['f1score'], df10['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    1c     3c   0.1105 0.3485   -0.08 0.3009  False
    1c     fc   0.0816 0.5556 -0.1089  0.272  False
    3c     fc  -0.0289    0.9  -0.264 0.2062  False
---------------------------------------------------


In [14]:
mc = MultiComparison(df30['f1score'], df30['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    1c     3c   0.1321 0.1936 -0.0488  0.313  False
    1c     fc   0.1179 0.1588 -0.0343 0.2702  False
    3c     fc  -0.0142    0.9 -0.2175 0.1892  False
---------------------------------------------------


In [15]:
mc = MultiComparison(df60['f1score'], df60['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    1c     3c   0.0833 0.4436 -0.0796 0.2463  False
    1c     fc   0.1709 0.0123  0.0316 0.3101   True
    3c     fc   0.0875 0.4452  -0.084  0.259  False
---------------------------------------------------


# Analysis <font color=blue>without </font> combining RF, DT and SVM into 1C 10 min

In [16]:
df = pd.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/skout.xlsx")
df = df[['time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']", "fc")
df.methods = df.methods.replace("['decision_tree', 'libsvm_svc', 'random_forest']", "3c")
df.methods = df.methods.replace("['libsvm_svc']", "svc")
df.methods = df.methods.replace("['decision_tree']", "dt")
df.methods = df.methods.replace("['random_forest']", "rf")
df10 = df[df.time_budget==10]
df30 = df[df.time_budget==30]
df60 = df[df.time_budget==60]
df.shape

(900, 3)

In [17]:
rp.summary_cont(df10['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,300.0,0.748181,0.241727,0.013956,0.720717,0.775646


In [18]:
rp.summary_cont(df30['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,300.0,0.77951,0.202243,0.011677,0.756531,0.802488


In [19]:
rp.summary_cont(df60['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,300.0,0.772674,0.198329,0.011451,0.75014,0.795208


In [20]:
stats.f_oneway(df10['f1score'][df10['methods'] == '1c'], 
             df10['f1score'][df10['methods'] == '3c'],
             df10['f1score'][df10['methods'] == 'fc'])

C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\scipy\stats\stats.py:3225: RuntimeWarning: invalid value encountered in double_scalars
  ssbn += _square_of_sums(a - offset) / len(a)


F_onewayResult(statistic=nan, pvalue=nan)

In [21]:
stats.f_oneway(df30['f1score'][df30['methods'] == '1c'], 
             df30['f1score'][df30['methods'] == '3c'],
             df30['f1score'][df30['methods'] == 'fc'])

F_onewayResult(statistic=nan, pvalue=nan)

In [22]:
stats.f_oneway(df60['f1score'][df60['methods'] == '1c'], 
             df60['f1score'][df60['methods'] == '3c'],
             df60['f1score'][df60['methods'] == 'fc'])

F_onewayResult(statistic=nan, pvalue=nan)

In [23]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model10 = ols('f1score ~ C(methods)', df10).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model10.df_model: .0f},{model10.df_resid: .0f}) = {model10.fvalue: .3f}, p = {model10.f_pvalue: .4f}")
    model10.summary()

Overall model F( 4, 295) =  5.433, p =  0.0003


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.069
Model:                            OLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                     5.433
Date:                Fri, 01 Nov 2019   Prob (F-statistic):           0.000311
Time:                        22:28:57   Log-Likelihood:                 11.466
No. Observations:                 300   AIC:                            -12.93
Df Residuals:                     295   BIC:                             5.587
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8037      0.030     26.505      0.000       0.744       0.863
C(methods)[T.dt]     -0.0791      0.043     -1.844      0.066      -0.163       0.005
C(methods)[T.fc]      0.0012      0.043      0.028      0.978      -0.083       0.086
C(methods)[T.rf]     -0.0318      0.043     -0.743      0.458      -0.116       0.053
C(methods)[T.svc]    -0.1676      0.043     -3.909      0.000      -0.252      -0.083
==============================================================================
Omnibus:                       62.558   Durbin-Watson:                   0.434
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               98.299
Skew:                          -1.246   Prob(JB):                     4.51e-22
Kurtosis:                       4.287   Cond. No.                         5.83
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model30 = ols('f1score ~ C(methods)', df30).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model30.df_model: .0f},{model30.df_resid: .0f}) = {model30.fvalue: .3f}, p = {model30.f_pvalue: .4f}")
    model30.summary()

Overall model F( 4, 295) =  3.543, p =  0.0077


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     3.543
Date:                Fri, 01 Nov 2019   Prob (F-statistic):            0.00766
Time:                        22:28:57   Log-Likelihood:                 61.342
No. Observations:                 300   AIC:                            -112.7
Df Residuals:                     295   BIC:                            -94.17
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8084      0.026     31.486      0.000       0.758       0.859
C(methods)[T.dt]     -0.0452      0.036     -1.244      0.214      -0.117       0.026
C(methods)[T.fc]      0.0017      0.036      0.047      0.963      -0.070       0.073
C(methods)[T.rf]      0.0058      0.036      0.160      0.873      -0.066       0.077
C(methods)[T.svc]    -0.1070      0.036     -2.947      0.003      -0.178      -0.036
==============================================================================
Omnibus:                       57.172   Durbin-Watson:                   0.495
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               87.881
Skew:                          -1.146   Prob(JB):                     8.26e-20
Kurtosis:                       4.332   Cond. No.                         5.83
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model60 = ols('f1score ~ C(methods)', df60).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model60.df_model: .0f},{model60.df_resid: .0f}) = {model60.fvalue: .3f}, p = {model60.f_pvalue: .4f}")
    model10.summary()

Overall model F( 4, 295) =  2.383, p =  0.0515


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.069
Model:                            OLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                     5.433
Date:                Fri, 01 Nov 2019   Prob (F-statistic):           0.000311
Time:                        22:28:57   Log-Likelihood:                 11.466
No. Observations:                 300   AIC:                            -12.93
Df Residuals:                     295   BIC:                             5.587
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8037      0.030     26.505      0.000       0.744       0.863
C(methods)[T.dt]     -0.0791      0.043     -1.844      0.066      -0.163       0.005
C(methods)[T.fc]      0.0012      0.043      0.028      0.978      -0.083       0.086
C(methods)[T.rf]     -0.0318      0.043     -0.743      0.458      -0.116       0.053
C(methods)[T.svc]    -0.1676      0.043     -3.909      0.000      -0.252      -0.083
==============================================================================
Omnibus:                       62.558   Durbin-Watson:                   0.434
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               98.299
Skew:                          -1.246   Prob(JB):                     4.51e-22
Kurtosis:                       4.287   Cond. No.                         5.83
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

mc = MultiComparison(df10['f1score'], df10['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
    3c     dt  -0.0791 0.3507 -0.1968  0.0386  False
    3c     fc   0.0012    0.9 -0.1165  0.1189  False
    3c     rf  -0.0318    0.9 -0.1495  0.0858  False
    3c    svc  -0.1676 0.0011 -0.2853 -0.0499   True
    dt     fc   0.0803  0.335 -0.0374   0.198  False
    dt     rf   0.0472 0.7803 -0.0705  0.1649  False
    dt    svc  -0.0885  0.238 -0.2062  0.0291  False
    fc     rf   -0.033    0.9 -0.1507  0.0847  False
    fc    svc  -0.1688  0.001 -0.2865 -0.0511   True
    rf    svc  -0.1358 0.0146 -0.2535 -0.0181   True
----------------------------------------------------


In [27]:
mc = MultiComparison(df30['f1score'], df30['methods'])
mc_results = mc.tukeyhsd(model30.f_pvalue)
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.01
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    3c     dt  -0.0452 0.6994 -0.1674  0.077  False
    3c     fc   0.0017    0.9 -0.1205 0.1239  False
    3c     rf   0.0058    0.9 -0.1164  0.128  False
    3c    svc   -0.107 0.0284 -0.2292 0.0152  False
    dt     fc   0.0469  0.673 -0.0753 0.1691  False
    dt     rf    0.051  0.609 -0.0712 0.1732  False
    dt    svc  -0.0618 0.4356  -0.184 0.0604  False
    fc     rf   0.0041    0.9 -0.1181 0.1263  False
    fc    svc  -0.1087 0.0247 -0.2309 0.0135  False
    rf    svc  -0.1128 0.0176  -0.235 0.0094  False
---------------------------------------------------


In [28]:
mc = MultiComparison(df60['f1score'], df60['methods'])
mc_results = mc.tukeyhsd(model60.f_pvalue)
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
    3c     dt  -0.0309    0.9 -0.1289  0.0672  False
    3c     fc   0.0296    0.9 -0.0685  0.1277  False
    3c     rf   -0.002    0.9 -0.1001  0.0961  False
    3c    svc  -0.0742  0.237 -0.1723  0.0239  False
    dt     fc   0.0605 0.4463 -0.0376  0.1585  False
    dt     rf   0.0289    0.9 -0.0692  0.1269  False
    dt    svc  -0.0433 0.7204 -0.1414  0.0548  False
    fc     rf  -0.0316    0.9 -0.1297  0.0665  False
    fc    svc  -0.1038 0.0332 -0.2019 -0.0057   True
    rf    svc  -0.0722 0.2629 -0.1703  0.0259  False
----------------------------------------------------
